In [10]:
# import required packages
import os
import pandas as pd
import numpy as np

In [25]:
# Start new Magic contribution for a locality

sites_dict = {'site': [], 'method_codes': [], 'citations': []}
samples_dict = {'site': [], 'sample': [],'method_codes': [], 'citations': [],
                'azimuth': [], 'dip': [], 'bed_dip_direction': [], 'bed_dip': [],'result_quality': []}
specimens_dict = {'specimen': [], 'sample': [],'method_codes': [],'experiments': [],'volume': [], 'citations': [],
                  'result_quality':[]}
measurement_dict = {'specimen': [],'method_codes': [],'dir_dec': [],'dir_inc': [],'magn_volume': [], 'experiment': [],
                    'dir_csd': [],'treat_temp':[], 'meas_temp': [],'treat_ac_field': [], 'treat_dc_field': [],'citations': [],
                     'treat_dc_field_phi': [],'instrument_codes': [],'sequence':[], 'magn_moment': [], 'measurement': [],
                    'treat_dc_field_theta': [],'treat_step_num':[],'quality': []}

### Cryo data

In [ ]:
# run this cell once for each file that you want to have in a location
# when done, skip to the very last cell that writes magic files
# .dat and .az files have to be in the same directory!! the name of the .az file has to agree with the one specified in the .dat file!

file_path = r'G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC/22Deccan_Thermal-Master.dat' #path + file name

treatment = 'both' # 'THD', 'AFD', or both' for directions; 'IZZI' for izzi pints (more to be added when needed)
min_THD_step = 100 # only used if treatment = 'both'

DC_field = 60 # uT value for the used lab field in intensity experiments
DC_phi = 0 # 'declination' of applied field 
DC_theta = 90 # 'inclination' of applied field 

site_delimiter = '-' # e.g., '-' for MAT1-1.2 ([site]-[sample].[specimen]); if there is none, use ''
site_chars = 3 # number of characters at the beginning of the sample name that are the site identifier. only used if 
               # site_delimiter = ''
specimen_delimiter = '' # e.g., '.' for MAT1-1.2 ([site]-[sample].[specimen]); if specimen = sample, use ''
specimen_chars = 0 # number of characters from the end of the sample names if there is no delimiter (use 0 for spec = sample)

#----------------------------------------------------------------------------------------------

f = open(file_path)
data = f.read().splitlines()
f.close
temp_data, specs = [],[]
if True: # remove line
    for i in range(len(data)):
        if i <= 4:
            temp_data.append(data[i])
        else:
            line = data[i].split()
            if line[0] in specs: continue
            specs.append(line[0])
            temp_data.append(data[i])
            for j in range(i+1,len(data)):
                line = data[j].split()
                if line[0] == specs[-1]: 
                    temp_data.append(data[j])
        #print(temp_data[-1])
    if len(temp_data) == len(data):
        data = temp_data
    else: 
        print('Measurement data could not be sorted')
        print(len(data),len(temp_data))
                

orientations = {'sample': [], 'azi': [], 'plunge': [], 'bed_dir': [], 'bed_dip': []}
sequence, last_treatment = 0, -1
for ii in range(len(data)):
    line = data[ii].split()
    if ii == 0:
        f = open(file_path.strip(line[0])+line[-1])
        orientations_raw = f.readlines()
        f.close()
        for o_line in orientations_raw:
            o_line = o_line.split()
            orientations['sample'].append(o_line[0])
            orientations['azi'].append(o_line[1])
            orientations['plunge'].append(o_line[2])
            orientations['bed_dir'].append(o_line[3])
            orientations['bed_dip'].append(o_line[4])
        continue
    elif ii < 5: continue
    
    if not site_delimiter == '':
        site_name = line[0].split(site_delimiter)[0]
    else:
        site_name = line[0][:site_chars+1]
    duplicate_site = False
    if not site_name in sites_dict['site']: sites_dict['site'].append(site_name)
    else: duplicate_site = True
    if not duplicate_site: sites_dict['citations'].append('This study')
    if treatment == 'THD': 
        if not duplicate_site: 
            sites_dict['method_codes'].append('LP-DIR-T')
        else: 
            for j in range(len(sites_dict['site'])):
                if sites_dict['site'] == site_name:
                    if not 'LP-DIR-T' in sites_dict['method_codes'][j]:
                        sites_dict['method_codes'][j] += ':LP-DIR-T'
    elif treatment == 'AFD': 
        if not duplicate_site: 
            sites_dict['method_codes'].append('LP-DIR-AF')
        else: 
            for j in range(len(sites_dict['site'])):
                if sites_dict['site'] == site_name:
                    if not 'LP-DIR-AF' in sites_dict['method_codes'][j]:
                        sites_dict['method_codes'][j] += ':LP-DIR-AF'
    elif treatment == 'both': 
        if not duplicate_site: 
            sites_dict['method_codes'].append('LP-DIR-AF:LP-DIR-T')
        else: 
            for j in range(len(sites_dict['site'])):
                if sites_dict['site'] == site_name:
                    if not 'LP-DIR-AF' in sites_dict['method_codes'][j]:
                        sites_dict['method_codes'][j] += ':LP-DIR-AF'
                    if not 'LP-DIR-T' in sites_dict['method_codes'][j]:
                        sites_dict['method_codes'][j] += ':LP-DIR-T'
    elif treatment == 'IZZI': 
        if not duplicate_site: 
            sites_dict['method_codes'].append('LP-PI-TRM:LP-PI-BT-IZZI:LP-PI-ALT-PTRM')
        else: 
            for j in range(len(sites_dict['site'])):
                if sites_dict['site'] == site_name:
                    if not 'LP-PI-BT-IZZI' in sites_dict['method_codes'][j]:
                        sites_dict['method_codes'][j] += ':LP-PI-TRM:LP-PI-BT-IZZI:LP-PI-ALT-PTRM'
    else: print('Currently, only THD (thermal demag - directions), AFD (AF demag - directions), both (combined AF+TH) and IZZI (izzi pint) are availabe options')
    
    name = line[0]
    if not specimen_delimiter == '':      
        sample_name = name.split(specimen_delimiter)[0]
        specimen_name = name
    else:       
        if not specimen_chars == 0:
            sample_name = name[:-specimen_chars]
        else:
            sample_name = name
        specimen_name = name
    
    duplicate_sample = False
    if not sample_name in samples_dict['sample']: samples_dict['sample'].append(sample_name)
    else: duplicate_sample = True
        
    if not duplicate_sample: 
        samples_dict['site'].append(site_name)
        samples_dict['citations'].append('This study')
        samples_dict['result_quality'].append('g')
    
    duplicate_specimen = False
    if not name in specimens_dict['specimen']:
        specimens_dict['specimen'].append(name)
        specimens_dict['sample'].append(sample_name)
        specimens_dict['citations'].append('This study')
        specimens_dict['result_quality'].append('g')
    else: duplicate_specimen = True
    
    if treatment == 'THD': 
        if not duplicate_sample: samples_dict['method_codes'].append('LP-DIR-T')
        else:
            for j in range(len(samples_dict['sample'])):
                if samples_dict['sample'][j] == sample_name:
                    if not 'LP-DIR-T' in samples_dict['method_codes'][j]:
                        samples_dict['method_codes'][j] += ':LP-DIR-T'
        if not duplicate_specimen:
            specimens_dict['method_codes'].append('LP-DIR-T')
            specimens_dict['experiments'].append(name+'_'+'LP-DIR-T')
    elif treatment == 'AFD': 
        if not duplicate_sample: samples_dict['method_codes'].append('LP-DIR-AF')
        else:
            for j in range(len(samples_dict['sample'])):
                if samples_dict['sample'][j] == sample_name:
                    if not 'LP-DIR-AF' in samples_dict['method_codes'][j]:
                        samples_dict['method_codes'][j] += ':LP-DIR-AF'
        if not duplicate_specimen:
            specimens_dict['method_codes'].append('LP-DIR-AF')
            specimens_dict['experiments'].append(name+'_'+'LP-DIR-AF')
    elif treatment == 'both': 
        if not duplicate_sample: samples_dict['method_codes'].append('LP-DIR-AF:LP-DIR-T')
        else:
            for j in range(len(samples_dict['sample'])):
                if samples_dict['sample'][j] == sample_name:
                    if not 'LP-DIR-T' in samples_dict['method_codes'][j] and 'LP-DIR-AF' in samples_dict['method_codes'][j]:
                        samples_dict['method_codes'][j] += ':LP-DIR-T'
                    elif not 'LP-DIR-AF' in samples_dict['method_codes'][j] and 'LP-DIR-T' in samples_dict['method_codes'][j]:
                        samples_dict['method_codes'][j] += ':LP-DIR-AF'
                    elif not 'LP-DIR-AF:LP-DIR-T' in samples_dict['method_codes'][j] and  not 'LP-DIR-T:LP-DIR-AF' in samples_dict['method_codes'][j]:
                        print(sample_name, samples_dict['method_codes'][j])
                        samples_dict['method_codes'][j] += ':LP-DIR-AF:LP-DIR-T'
                    break
        if not duplicate_specimen:
            specimens_dict['method_codes'].append('LP-DIR-AF:LP-DIR-T')
            specimens_dict['experiments'].append(name+'_'+'LP-DIR-AFT')
        
    elif treatment == 'IZZI':
        if not duplicate_sample: samples_dict['method_codes'].append('LP-PI-TRM:LP-PI-BT-IZZI:LP-PI-ALT-PTRM')
        else:
            for j in range(len(samples_dict['sample'])):
                if samples_dict['sample'][j] == sample_name:
                    if not 'LP-PI-BT-IZZI' in samples_dict['method_codes'][j]:
                        samples_dict['method_codes'][j] += ':LP-PI-TRM:LP-PI-BT-IZZI:LP-PI-ALT-PTRM'
        if not duplicate_specimen:
            specimens_dict['method_codes'].append('LP-PI-TRM:LP-PI-BT-IZZI:LP-PI-ALT-PTRM')
            specimens_dict['experiments'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
    
    
    
    if not duplicate_sample:
        for jj in range(len(orientations['sample'])):
            if orientations['sample'][jj] == name:
                samples_dict['azimuth'].append(str(orientations['azi'][jj]))
                samples_dict['dip'].append(str((float(orientations['plunge'][jj])-90)))

                samples_dict['bed_dip_direction'].append(str(orientations['bed_dir'][jj]))
                samples_dict['bed_dip'].append(str(orientations['bed_dip'][jj]))
                break
        if not len(samples_dict['sample']) == len(samples_dict['bed_dip']):
            print('Could not find orientation data for ', name)
            samples_dict['azimuth'].append('')
            samples_dict['dip'].append('') #check orientation
            samples_dict['bed_dip_direction'].append('')
            samples_dict['bed_dip'].append('')
    
    treat = float(line[1])
    volume = 10 * 10**-6    #  10 cc assumed by cryo to m2                                     
    if not duplicate_specimen: specimens_dict['volume'].append(str(volume)) # m^3
      
    measurement_dict['specimen'].append(name)
    
    if treat > last_treatment and not treatment == 'IZZI': sequence += 1
    elif treatment == 'IZZI': 
        sequence += 1
        if sequence > 1 and not measurement_dict['specimen'][-1] == measurement_dict['specimen'][-2]: sequence = 0
    if sequence < 10: seq_str = '0'+str(sequence)
    else: seq_str = str(sequence)
    measurement_dict['sequence'].append(str(seq_str))
    measurement_dict['citations'].append('This study')
    
    if sequence == 1: measurement_dict['quality'].append('g')
    else:
        if not treatment == 'IZZI':
            if treat == last_treatment: # attempting to catch repeat measurements
                #print(treat,last_treatment)
                measurement_dict['quality'][-1] = 'b' # assumes that the first one was bad
        measurement_dict['quality'].append('g')
                                             
    if treatment == 'THD':
        measurement_dict['treat_temp'].append(str(float(treat)+273)) # K
        measurement_dict['meas_temp'].append(str(273)) # K
        measurement_dict['treat_ac_field'].append(str(0)) # T
        measurement_dict['treat_dc_field'].append(str(0)) # T
        measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
        measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
        measurement_dict['method_codes'].append('LT-T-Z')
        measurement_dict['experiment'].append(name+'_'+'LP-DIR-T')
        measurement_dict['measurement'].append(name+'_'+'LP-DIR-T'+'-'+str(seq_str))
                    
    elif treatment == 'AFD':
        measurement_dict['treat_temp'].append(str(273)) # K
        measurement_dict['meas_temp'].append(str(273)) # K
        measurement_dict['treat_ac_field'].append(str(float(treat)/10**3)) # T
        measurement_dict['treat_dc_field'].append(str(0)) # T
        measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
        measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
        measurement_dict['method_codes'].append('LT-AF-Z')
        measurement_dict['experiment'].append(name+'_'+'LP-DIR-AF')
        measurement_dict['measurement'].append(name+'_'+'LP-DIR-AF'+'-'+str(seq_str))
    
    elif treatment == 'both':
        if treat < min_THD_step:
            measurement_dict['treat_temp'].append(str(273)) # K
            measurement_dict['meas_temp'].append(str(273)) # K
            measurement_dict['treat_ac_field'].append(str(float(treat)/10**3)) # T
            measurement_dict['treat_dc_field'].append(str(0)) # T
            measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
            measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
            measurement_dict['method_codes'].append('LT-AF-Z')
            measurement_dict['experiment'].append(name+'_'+'LP-DIR-AFT')
            measurement_dict['measurement'].append(name+'_'+'LP-DIR-AFT'+'-'+str(seq_str))
        else:
            measurement_dict['treat_temp'].append(str(float(treat)+273)) # K
            measurement_dict['meas_temp'].append(str(273)) # K
            measurement_dict['treat_ac_field'].append(str(0)) # T
            measurement_dict['treat_dc_field'].append(str(0)) # T
            measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
            measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
            measurement_dict['method_codes'].append('LT-T-Z')
            measurement_dict['experiment'].append(name+'_'+'LP-DIR-AFT')
            measurement_dict['measurement'].append(name+'_'+'LP-DIR-AFT'+'-'+str(seq_str))
            
    if treatment == 'IZZI':
        if str(treat)[-2:] == '.0': # Z
            measurement_dict['treat_temp'].append(str(float(str(treat)[:-2])+273)) # K
            measurement_dict['meas_temp'].append(str(273)) # K
            measurement_dict['treat_dc_field'].append(str(0)) # T
            measurement_dict['treat_ac_field'].append(str(0)) # T
            measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
            measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
            if treat == 0:
                measurement_dict['method_codes'].append('LT-NO:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            else:
                if 'LT-NO' in measurement_dict['method_codes'][-1] or 'LT-PTRM-I' in measurement_dict['method_codes'][-1] \
                or ('LT-T-Z' in measurement_dict['method_codes'][-1] and 'LP-PI-TRM-IZ' in measurement_dict['method_codes'][-1]):
                    measurement_dict['method_codes'].append('LT-T-Z:LP-PI-TRM-ZI:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
                else:
                    measurement_dict['method_codes'].append('LT-T-Z:LP-PI-TRM-IZ:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            measurement_dict['experiment'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement_dict['measurement'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(treat)[-2:] == '.1': # I
            measurement_dict['treat_temp'].append(str(float(str(treat)[:-2])+273)) # K
            measurement_dict['meas_temp'].append(str(273)) # K
            measurement_dict['treat_dc_field'].append(str(DC_field*10**-6)) # T
            measurement_dict['treat_ac_field'].append(str(0)) # T
            measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
            measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
            if 'LT-PTRM-I' in measurement_dict['method_codes'][-1] \
                or ('LT-T-Z' in measurement_dict['method_codes'][-1] and 'LP-PI-TRM-ZI' in measurement_dict['method_codes'][-1]):
                measurement_dict['method_codes'].append('LT-T-I:LP-PI-TRM-ZI:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            else:
                measurement_dict['method_codes'].append('LT-T-I:LP-PI-TRM-IZ:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            measurement_dict['experiment'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement_dict['measurement'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(treat)[-2:] == '.2': # P
            measurement_dict['treat_temp'].append(str(float(str(treat)[:-2])+273)) # K
            measurement_dict['meas_temp'].append(str(273)) # K
            measurement_dict['treat_dc_field'].append(str(DC_field*10**-6)) # T
            measurement_dict['treat_ac_field'].append(str(0)) # T
            measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
            measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
            measurement_dict['method_codes'].append('LT-PTRM-I:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            measurement_dict['experiment'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement_dict['measurement'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(treat)[-2:] == '.3': # T
            measurement_dict['treat_temp'].append(str(float(str(treat)[:-2])+273)) # K
            measurement_dict['meas_temp'].append(str(273)) # K
            measurement_dict['treat_dc_field'].append(str(0)) # T
            measurement_dict['treat_ac_field'].append(str(0)) # T
            measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
            measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
            measurement_dict['method_codes'].append('LT-PTRM-Z:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            measurement_dict['experiment'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement_dict['measurement'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(treat)[-2:] == '.4': # A
            measurement_dict['treat_temp'].append(str(float(str(treat)[:-2])+273)) # K
            measurement_dict['meas_temp'].append(str(273)) # K
            measurement_dict['treat_dc_field'].append(str(0)) # T
            measurement_dict['treat_ac_field'].append(str(0)) # T
            measurement_dict['treat_dc_field_phi'].append(str(0)) # deg
            measurement_dict['treat_dc_field_theta'].append(str(0)) # deg
            measurement_dict['method_codes'].append('LT-PTRM-AC:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            measurement_dict['experiment'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement_dict['measurement'].append(name+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        
    intensity = float(line[4])*25/np.pi # Oe => Oe * 20/pi = A/m 
    measurement_dict['magn_volume'].append(str(float(intensity))) # A/m
    measurement_dict['magn_moment'].append(str((float(intensity))*(float(volume))))#/10**6))) # Am^2
    GDec = (line[7]) # not needed for demag GUI
    GInc = (line[8]) # not needed for demag GUI
    csd = line[3] # not sure if this is actually csd or something else
    measurement_dict['dir_csd'].append(csd)
    CDec = line[5]
    measurement_dict['dir_dec'].append(CDec)
    CInc = line[6]
    measurement_dict['dir_inc'].append(CInc)
    measurement_dict['instrument_codes'].append('Cryo')
    measurement_dict['treat_step_num'].append(str(int(seq_str)+1))
    last_treatment = treat
    
print('Done!')

#for key in samples_dict.keys():
#    print(key,len(samples_dict[key]))

### Molspin data files:

In [ ]:
# add measurement data for a site 
# run this cell once for each directory specified below. Use one directory per site & experiment
#assumes that specimen == sample

site_name = 'I1938' # Specify name of site
treatment = 'THD' # Specify if the experiment was thermal demag ('THD') or AF demag ('AFD')
site_dir_path = r'G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC\I1938' # absolute or relative path to directory 

#---------------------------------------------------------------------------------------------

duplicate_site = False
if not site_name in sites_dict['site']: sites_dict['site'].append(site_name)
else: duplicate_site = True
if not duplicate_site: sites_dict['citations'].append('This study')
if treatment == 'THD': 
    if not duplicate_site: 
        sites_dict['method_codes'].append('LP-DIR-T')
    else: 
        for j in range(len(sites_dict['site'])):
            if sites_dict['site'] == site_name:
                if not 'LP-DIR-T' in sites_dict['method_codes'][j]:
                    sites_dict['method_codes'][j] += ':LP-DIR-T'
elif treatment == 'AFD': 
    if not duplicate_site: 
        sites_dict['method_codes'].append('LP-DIR-AF')
    else: 
        for j in range(len(sites_dict['site'])):
            if sites_dict['site'] == site_name:
                if not 'LP-DIR-AF' in sites_dict['method_codes'][j]:
                    sites_dict['method_codes'][j] += ':LP-DIR-AF'
else: print('Currently, only THD (thermal demag - directions) or AFD (AF demag - directions) are availabe options')

if not site_dir_path.endswith('/'): site_dir_path += '/'
for root, dirs, files in os.walk(site_dir_path): 
    for file in files:
        with open(root+os.sep+file) as specimen_file:
            measurement_data = specimen_file.readlines()
        sequence, last_treat = 0, -1
        for i in range(len(measurement_data)):
            measurement_data[i] = measurement_data[i].strip('\n')
            if i == 0:
                samples_dict['site'].append(site_name)
                samples_dict['citations'].append('This study')
                samples_dict['result_quality'].append('g')
                specimens_dict['citations'].append('This study')
                specimens_dict['result_quality'].append('g')
                header = measurement_data[i].split()
                name = header[0]
                if treatment == 'THD': 
                    samples_dict['method_codes'].append('LP-DIR-T')
                    specimens_dict['method_codes'].append('LP-DIR-T')
                    specimens_dict['experiments'].append(name+'_'+'LP-DIR-T')
                elif treatment == 'AFD': 
                    samples_dict['method_codes'].append('LP-DIR-AF')
                    specimens_dict['method_codes'].append('LP-DIR-AF')
                    specimens_dict['experiments'].append(name+'_'+'LP-DIR-AF')
                samples_dict['sample'].append(name)
                specimens_dict['sample'].append(name)
                specimens_dict['specimen'].append(name)
                bearing = header[1]
                samples_dict['azimuth'].append(bearing)
                plunge = header[2]
                samples_dict['dip'].append(str(float(plunge)-90))
                strike = header[3]
                samples_dict['bed_dip_direction'].append(str(float(strike)+90))
                dip = header[4]
                samples_dict['bed_dip'].append(dip)
                volume = header[5]
                specimens_dict['volume'].append(str(float(volume)/10**6)) # m^3
            else:
                measurement = measurement_data[i].split()
                measurement_dict['citations'].append('This study')
                measurement_dict['specimen'].append(name)
                treat = int(measurement[0])
                if treatment == 'THD':
                    measurement_dict['treat_temp'].append(str(float(treat)+273)) # K
                    measurement_dict['meas_temp'].append(str(273)) # K
                    measurement_dict['treat_ac_field'].append(str(0)) # T
                    measurement_dict['method_codes'].append('LT-T-Z')
                    measurement_dict['experiment'].append(name+'_'+'LP-DIR-T')
                    measurement_dict['measurement'].append(name+'_'+'LP-DIR-T'+'-'+str(i))
                    
                elif treatment == 'AFD':
                    measurement_dict['treat_temp'].append(str(273)) # K
                    measurement_dict['meas_temp'].append(str(273)) # K
                    measurement_dict['treat_ac_field'].append(str(float(treat)/10**3)) # T
                    measurement_dict['method_codes'].append('LT-AF-Z')
                    measurement_dict['experiment'].append(name+'_'+'LP-DIR-AF')
                    measurement_dict['measurement'].append(name+'_'+'LP-DIR-AF'+'-'+str(i))

                if treat > last_treat: sequence += 1
                if sequence == 1:
                    measurement_dict['quality'].append('g')
                else:
                        if not treat > last_treat: # attempting to catch repeat measurements
                            measurement_dict['quality'][-1] = 'b' # assumes that the first one was bad
                        measurement_dict['quality'].append('g')
                    
                if sequence < 10: seq_str = '0'+str(sequence)
                else: seq_str = str(sequence)
                measurement_dict['sequence'].append(seq_str)
                intensity = measurement[1] # mA/m
                measurement_dict['magn_volume'].append(str(float(intensity)/10**3)) # A/m
                measurement_dict['magn_moment'].append(str((float(intensity)/10**3)*(float(volume)/10**6))) # Am^2
                GDec = (measurement[2]) # not needed for demag GUI
                GInc = (measurement[3]) # not needed for demag GUI
                a95 = measurement[4]
                measurement_dict['dir_csd'].append(a95)
                CDec = measurement[5]
                measurement_dict['dir_dec'].append(CDec)
                CInc = measurement[6]
                measurement_dict['dir_inc'].append(CInc)
                measurement_dict['instrument_codes'].append('Molspin')
                measurement_dict['treat_dc_field'].append(0)
                measurement_dict['treat_dc_field_theta'].append(0)
                measurement_dict['treat_dc_field_phi'].append(0)
                measurement_dict['treat_step_num'].append(seq_str)

print('Specimens for site', site_name, 'added successfully')
#for key in measurement_dict.keys():
#    print(key,len(measurement_dict[key]))

## JR6 data

convert jr6 data to magic format. Experimental. Agico orientation parameters are not considered at the moment. Use with caution!

In [26]:
path = r'G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC' #os.getcwd()
file = 'test_data.jr6'
volume = 7 #cc

protocol = 'Dir' # 'PTSD' 'Dir'
# for pseuro-Thellier and Shaw experiments: ARM demag series have to have .01 as identifier in AF field

DC_field = 60 # uT value for the used lab field in intensity experiments
DC_phi = 0 # 'declination' of applied field 
DC_theta = 90 # 'inclination' of applied field 

site_delimiter = '-' # e.g., '-' for MAT1-1.2 ([site]-[sample].[specimen]); if there is none, use ''
site_chars = 2 # number of characters at the beginning of the sample name that are the site identifier. only used if 
               # site_delimiter = ''
specimen_delimiter = '' # e.g., '.' for MAT1-1.2 ([site]-[sample].[specimen]); if specimen = sample, use ''
specimen_chars = 1 # number of characters from the end of the sample names if there is no delimiter (use 0 for spec = sample)

#---------------------------------------------------------------------------------------------


if not path.endswith('/'): path += '/'
f=open(path+file)
tempdata=f.readlines()
f.close()
tempfile = open('temp.txt','w')
tempfile.close()
for line in tempdata:
    line = line[:9].strip(' ') + line[9:].replace('-',',-').replace(' ',',').replace('\t',',')
    while ',,' in line:
        line=line.replace(',,',',')
    with open('temp.txt','a') as tempfile:
        tempfile.write(line)
jr6_raw = pd.read_csv('temp.txt',sep=",",header=None,names=['specimen','treat','m_x','m_y','m_z','exp','azimuth','spec_plunge',
                                                            'tect_azi','tect_plunge','bed_dip_direction','bed_plunge','orient_1',
                                                            'orient_2','orient_3','orient_4','csd'])
os.remove('temp.txt')
jr6_raw['m_x'] = (jr6_raw['m_x'].astype(str)+'E'+jr6_raw['exp'].astype(str)).astype(float)
jr6_raw['m_y'] = (jr6_raw['m_y'].astype(str)+'E'+jr6_raw['exp'].astype(str)).astype(float)
jr6_raw['m_z'] = (jr6_raw['m_z'].astype(str)+'E'+jr6_raw['exp'].astype(str)).astype(float)
jr6_raw['magn_volume'] = np.sqrt(jr6_raw['m_x']**2 + jr6_raw['m_y']**2 + jr6_raw['m_z']**2) # A/m
jr6_raw['magn_moment'] = jr6_raw['magn_volume']*(float(volume)/10**6) # Am2
jr6_raw['dir_dec'] = round((np.arctan2(jr6_raw['m_y'],jr6_raw['m_x'])*180/np.pi)%360,2)
jr6_raw['dir_inc'] = round((np.arctan2(jr6_raw['m_z'],np.sqrt(jr6_raw['m_x']**2+jr6_raw['m_y']**2)) *180/np.pi),2) # A/m
#print(jr6_raw)
if not sum(jr6_raw['tect_azi']) == 0 and not sum(jr6_raw['tect_plunge']) == 0: 
    samples_dict['tect_azi'] = []
    samples_dict['tect_plunge'] = []
jr6_raw['dir_csd'] = np.zeros(len(jr6_raw))
jr6_raw['meas_temp'] = np.ones(len(jr6_raw))*273
jr6_raw['volume'] = np.ones(len(jr6_raw))*float(volume)/10**6
jr6_raw['dip'] = jr6_raw['spec_plunge']-90
jr6_raw['instrument_codes'] = 'JR6'
jr6_raw['quality'] = 'g'
jr6_raw['result_quality'] = 'g'
jr6_raw['citations'] = 'This study'
jr6_raw['bed_dip'] = jr6_raw['bed_plunge']#-90
jr6_raw['treat_dc_field'] = np.zeros(len(jr6_raw))#np.ones(len(jr6_raw)) * DC_field *10**-6
jr6_raw['treat_dc_field_phi'] = np.zeros(len(jr6_raw))#np.ones(len(jr6_raw)) * DC_phi
jr6_raw['treat_dc_field_theta'] = np.zeros(len(jr6_raw))#np.ones(len(jr6_raw)) * DC_theta

jr6_raw['index'] = jr6_raw.index
jr6_raw = jr6_raw.sort_values(by=['specimen','index'])
site, sample, method_code, treat_ac_field, experiment,treat_temp, measurement = [],[],[],[],[],[],[]
sequence, treat_step_num = [],[]
if not protocol == 'Dir': treat_dc_field, treat_dc_field_phi, treat_dc_field_theta = [],[],[]
lastname = ''
for i in range(len(jr6_raw)):
    if i == 0: lastname = jr6_raw.loc[jr6_raw.index[i],'specimen']
    if not site_delimiter == '': 
        site.append(jr6_raw.loc[jr6_raw.index[i],'specimen'].split(site_delimiter)[0])
    else: 
        site.append(jr6_raw.loc[jr6_raw.index[i],'specimen'][:site_chars+1])
                       
    if not specimen_delimiter == '': sample.append(''.join(jr6_raw.loc[jr6_raw.index[i],'specimen'].split(site_delimiter)[:-1]))
    else:
        if not specimen_chars == 0:
            sample.append(jr6_raw.loc[jr6_raw.index[i],'specimen'][:-specimen_chars])
        else:
            sample.append(jr6_raw.loc[jr6_raw.index[i],'specimen'])
            
    if i > 0 and lastname == jr6_raw.loc[jr6_raw.index[i],'specimen']:
        sequence.append(int(sequence[-1])+1)
    else:
        sequence.append(0)
    if sequence[-1] < 10: seq_str = '00'+str(sequence[-1])
    elif i < 100: seq_str = '0'+str(sequence[-1])
    else: seq_str = str(sequence[-1])
    treat_step_num.append(int(seq_str))
    sequence[-1] = seq_str
                        
    if protocol == 'Dir':
        if 'NRM' in jr6_raw.loc[jr6_raw.index[i],'treat']:
            method_code.append('LT-AF-Z') 
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-AF') 
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-AF'+'-'+str(i)) 
            treat_temp.append(273)
            treat_ac_field.append(0)
            #check for double NRM steps
            if i < len(jr6_raw):
                if 'NRM' in jr6_raw.loc[jr6_raw.index[i+1],'treat']:
                    for j in range(i,len(jr6_raw)):
                        if not 'NRM' in jr6_raw.loc[jr6_raw.index[j],'treat']:
                            if 'TD' in jr6_raw.loc[jr6_raw.index[j],'treat']:
                                method_code[-1] = 'LT-T-Z'
                                experiment[-1] = str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-T'
                                measurement[-1] = str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-T'+'-'+str(i)
                                break
                            elif 'AD' in jr6_raw.loc[jr6_raw.index[j],'treat']:
                                break
        if 'AD' in jr6_raw.loc[jr6_raw.index[i],'treat']: 
            method_code.append('LT-AF-Z')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-AF')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-AF'+'-'+str(i))
            treat_temp.append(273)
            treat_ac_field.append(float(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('AD'))*10**-3)
        elif 'TD' in jr6_raw.loc[jr6_raw.index[i],'treat']: 
            method_code.append('LT-T-Z')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-T')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-T'+'-'+str(i))
            treat_temp.append(float(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('TD'))+273)
            treat_ac_field.append(0)
            if 'NRM' in jr6_raw.loc[jr6_raw.index[i-1],'treat']:
                method_code[-2] = 'LT-T-Z'
                experiment[-2] = str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-T'
                measurement[-2] = str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-DIR-T'+'-'+str(i)
    elif protocol == 'PTSD':
        if 'NRM' in jr6_raw.loc[jr6_raw.index[i],'treat']: 
            method_code.append('LT-NO:LP-DIR-AF:LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM'+'-'+str(i))
            treat_temp.append(273)
            treat_ac_field.append(0)
            treat_dc_field.append(str(0*DC_field*10**-6))
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
        if 'AD' in jr6_raw.loc[jr6_raw.index[i],'treat']: 
            if not jr6_raw.loc[jr6_raw.index[i],'treat'].endswith('.01'):
                method_code.append('LT-AF-Z:LP-DIR-AF:LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
                treat_temp.append(273)
                treat_ac_field.append(float(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('AD'))*10**-3)
                experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
                measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM'+'-'+str(i))
                treat_dc_field.append(str(0*DC_field*10**-6))
                treat_dc_field_phi.append(str(DC_phi))
                treat_dc_field_theta.append(str(DC_theta))
            else:
                method_code.append('LT-AF-Z:LP-ARM-AFD:LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
                treat_temp.append(273)
                treat_ac_field.append(float(jr6_raw.loc[jr6_raw.index[i],'treat'][:-1].strip('AD'))*10**-3)
                experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
                measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM'+'-'+str(i))
                treat_dc_field.append(str(0*DC_field*10**-6))
                treat_dc_field_phi.append(str(DC_phi))
                treat_dc_field_theta.append(str(DC_theta))

        elif 'TD' in jr6_raw.loc[jr6_raw.index[i],'treat']: 
            method_code.append('LT-T-I:LP-TRM-AFD:LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
            treat_temp.append(float(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('TD'))+273)
            treat_ac_field.append(0)
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM'+'-'+str(i))
            treat_dc_field.append(str(DC_field*10**-6))
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
        elif 'ARM' in jr6_raw.loc[jr6_raw.index[i],'treat']: 
            method_code.append('LT-AF-I:LP-ARM-AFD:LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
            treat_temp.append(273)
            treat_ac_field.append(float(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('ARM'))*10**-3)
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM:LP-PI-ARM:LP-PI-ALT-AFARM'+'-'+str(i))
            treat_dc_field.append(str(0*DC_field*10**-6))
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
    elif protocol == 'IZZI':
        #print(jr6_raw.loc[jr6_raw.index[i],'treat'])
        if 'NRM' in jr6_raw.loc[jr6_raw.index[i],'treat']: 
            method_code.append('LT-NO:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(i))
            treat_temp.append(273)
            treat_ac_field.append(0)
            treat_dc_field.append(str(0*DC_field*10**-6))
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
        elif str(jr6_raw.loc[jr6_raw.index[i],'treat'])[-2:] == '.0': # Z
            treat_temp.append(str(float(str(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('TD'))[:-2])+273)) # K
            treat_dc_field.append(str(0*DC_field*10**-6))
            treat_ac_field.append(str(0)) # T
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
            if 'LT-NO' in method_code[-1] or 'LT-PTRM-I' in method_code[-1] \
                or ('LT-T-Z' in method_code[-1] and 'LP-PI-TRM-IZ' in method_code[-1]):
                    method_code.append('LT-T-Z:LP-PI-TRM-ZI:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            else:
                    method_code.append('LT-T-Z:LP-PI-TRM-IZ:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(jr6_raw.loc[jr6_raw.index[i],'treat'])[-2:] == '.1': # I
            treat_temp.append(str(float(str(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('TD'))[:-2])+273)) # K
            treat_dc_field.append(str(DC_field*10**-6))
            treat_ac_field.append(str(0)) # T
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
            if 'LT-PTRM-I' in method_code[-1] \
                or ('LT-T-Z' in method_code[-1] and 'LP-PI-TRM-ZI' in method_code[-1]):
                method_code.append('LT-T-I:LP-PI-TRM-ZI:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            else:
                method_code.append('LT-T-I:LP-PI-TRM-IZ:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(jr6_raw.loc[jr6_raw.index[i],'treat'])[-2:] == '.2': # P
            treat_temp.append(str(float(str(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('TD'))[:-2])+273)) # K
            treat_dc_field.append(str(DC_field*10**-6))
            treat_ac_field.append(str(0)) # T
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
            method_code.append('LT-PTRM-I:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(jr6_raw.loc[jr6_raw.index[i],'treat'])[-2:] == '.3': # T
            treat_temp.append(str(float(str(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('TD'))[:-2])+273)) # K
            treat_dc_field.append(str(0*DC_field*10**-6))
            treat_ac_field.append(str(0)) # T
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
            method_code.append('LT-PTRM-Z:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
        elif str(jr6_raw.loc[jr6_raw.index[i],'treat'])[-2:] == '.4': # A
            treat_temp.append(str(float(str(jr6_raw.loc[jr6_raw.index[i],'treat'].strip('TD'))[:-2])+273)) # K
            treat_dc_field.append(str(0*DC_field*10**-6))
            treat_ac_field.append(str(0)) # T
            treat_dc_field_phi.append(str(DC_phi))
            treat_dc_field_theta.append(str(DC_theta))
            method_code.append('LT-PTRM-AC:LP-PI-TRM:LP-PI-ALT-PTRM:LP-PI-BT-IZZI')
            experiment.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI')
            measurement.append(str(jr6_raw.loc[jr6_raw.index[i],'specimen'])+'_'+'LP-PI-TRM_LP-PI-ALT-PTRM_LP-PI-BT-IZZI'+'-'+str(seq_str))
    else:
        print('protocol has not been added yet. contact whoever is responsible for this notebook')
    lastname = jr6_raw.loc[jr6_raw.index[i],'specimen']    
jr6_raw['site'] = site
jr6_raw['sample'] = sample
jr6_raw['method_codes'] = method_code
jr6_raw['experiment'] = experiment
jr6_raw['measurement'] = measurement
jr6_raw['sequence'] = sequence
jr6_raw['treat_step_num'] = treat_step_num
jr6_raw['treat_temp'] = treat_temp
jr6_raw['treat_ac_field'] = treat_ac_field
if not protocol == 'Dir':
    jr6_raw['treat_dc_field'] = treat_dc_field
    jr6_raw['treat_dc_field_phi'] = treat_dc_field_phi
    jr6_raw['treat_dc_field_theta'] = treat_dc_field_theta
                                                   
sites_new = list(set(jr6_raw['site']))
for i in range(len(sites_new)):
    if not sites_new[i] in sites_dict['site']:
        sites_dict['site'].append(sites_new[i])
for i in range(len(sites_dict['site'])):
    temp_df = jr6_raw[jr6_raw['site'] == sites_dict['site'][i]]
    if not len(temp_df) == 0:
        sites_dict['method_codes'].append(':'.join(list(set(temp_df['method_codes'].astype(str)))))
        sites_dict['citations'].append(':'.join(list(set(temp_df['citations'].astype(str)))))
samples_new = list(set(jr6_raw['sample']))
for i in range(len(samples_new)):
    if not samples_new[i] in samples_dict:
        samples_dict['sample'].append(samples_new[i])
for i in range(len(samples_dict['sample'])):
    temp_df = jr6_raw[jr6_raw['sample'] == samples_dict['sample'][i]]
    if not len(temp_df) == 0:
        for key in samples_dict.keys():
            if not key == 'sample':
                samples_dict[key].append(':'.join(list(set(temp_df[key].astype(str)))))
specimens_new = list(set(jr6_raw['specimen']))
for i in range(len(specimens_new)):
    if not specimens_new[i] in specimens_dict:
        specimens_dict['specimen'].append(specimens_new[i])
for i in range(len(specimens_dict['specimen'])):
    temp_df = jr6_raw[jr6_raw['specimen'] == specimens_dict['specimen'][i]]
    if not len(temp_df) == 0:
        for key in specimens_dict.keys():
            if key == 'experiments':
                specimens_dict[key].append(':'.join(list(set(temp_df['experiment'].astype(str)))))
            elif not key == 'specimen':
                specimens_dict[key].append(':'.join(list(set(temp_df[key].astype(str)))))

for key in measurement_dict.keys():
    measurement_dict[key] = jr6_raw[key].values
print('Done!')

Done!


In [ ]:
# this cell only exists to find issues with the notebook
for key in measurement_dict.keys():
    print(measurement_dict[key], len(measurement_dict[key]))

### Write MagIC formatted files

Run cell after all experiments of the current location have been added using the cell above 

In [27]:
# save files for demag_gui
savepath = './'

if not savepath.endswith('/'): savepath += '/'
with open(savepath+'sites.txt', 'w') as sites_file:
    sites_file.write('tab\tsites\n')
sites_df = pd.DataFrame(sites_dict)
sites_df.to_csv(savepath+'sites.txt',mode='a',index=False,sep='\t')
    
with open(savepath+'samples.txt', 'w') as samples_file:
    samples_file.write('tab\tsamples\n')
samples_df = pd.DataFrame(samples_dict)
samples_df.to_csv(savepath+'samples.txt',mode='a',index=False,sep='\t')

with open(savepath+'specimens.txt', 'w') as specimens_file:
    specimens_file.write('tab\tspecimens\n')
specimens_df = pd.DataFrame(specimens_dict)
specimens_df.to_csv(savepath+'specimens.txt',mode='a',index=False,sep='\t')

with open(savepath+'measurements.txt', 'w') as measurements_file:
    measurements_file.write('tab\tmeasurements\n')
measurement_df = pd.DataFrame(measurement_dict)
measurement_df = measurement_df[(~measurement_df['specimen'].isna())]
#measurement_df = measurement_df.sort_values(by=['specimen','sequence'])
measurement_df.to_csv(savepath+'measurements.txt',mode='a',index=False,sep='\t')

print('Files for pmagpy GUI saved to', os.getcwd())

Files for pmagpy GUI saved to G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC\UF2MagIC


# Combine Magic files of multiple batches:

Needs more testing!

In [ ]:
# give the links to the directories where measurements, specimens, samples, sites (etc) files are located
links_to_individual_batches = {
    r'G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC\Testdata\PI-IZZI',
    r'G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC\Testdata\PI-PTSD'
}
save_dir = './' #link to where the combined magic files should be saved


#---------------------------------------------------------------------------------------------


try:
    del measurements_df
    del specimens_df
    del samples_df
    del sites_df
    if not save_dir.endswith('/'): save_dir += '/'
except: 
    if not save_dir.endswith('/'): save_dir += '/'


for link in links_to_individual_batches:
    if not link.endswith('/'): link += '/'
    print('! Working on batch in '+link)
    for magicfile in os.listdir(link):
        if magicfile == 'sites.txt':
            
            try: 
                #sites_dict = {'site': [], 'method_codes': [], 'citations': []}
                unique_sites = list(set(sites_df['site'])).astype(str) # check if df exists
                temp_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))
                if len(temp_df[temp_df['sites'].isin(unique_sites)]) == 0:
                    sites_df = pd.concat([sites_df,temp_df],ignore_index=True)
                else: 
                    sites_df = pd.concat([sites_df,temp_df[~temp_df['sites'].isin(unique_sites)]],ignore_index=True)
                    temp_df = temp_df[temp_df['sites'].isin(unique_sites)]                
                    
                    for key in ['method_codes','citations']:
                        for i in range(len(temp_df)):
                            if not temp_df.loc[temp_df.index[i],key] in sites_df.loc[sites_df['site']==temp_df.loc[temp_df.index[i],'site'],[key]]:
                                sites_df.loc[sites_df['site']==temp_df.loc[temp_df.index[i],'site',[key]]] += ':'+temp_df.loc[temp_df.index[i],key]

            except: # first batch -> dataframe
                sites_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))                    

           
        elif magicfile == 'samples.txt':
            #samples_dict = {'site': [], 'sample': [],'method_codes': [], 'citations': [],
            #    'azimuth': [], 'dip': [], 'bed_dip_direction': [], 'bed_dip': [],'result_quality': []}
            try:
                unique_samples = list(set(samples_df['sample'])) # check if df exists
                temp_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))
                if len(temp_df[temp_df['sample'].isin(unique_samples)]) == 0:
                    samples_df = pd.concat([samples_df,temp_df],ignore_index=True)
                else: 
                    samples_df = pd.concat([samples_df,temp_df[~temp_df['sample'].isin(unique_samples)]],ignore_index=True)
                    temp_df = temp_df[temp_df['sample'].isin(unique_samples)]                
                    
                    for key in ['method_codes','citations']:
                        for i in range(len(temp_df)):
                            if not temp_df.loc[temp_df.index[i],key] in samples_df.loc[samples_df['sample']==temp_df.loc[temp_df.index[i],'sample'],[key]]:
                                samples_df.loc[samples_df['sample']==temp_df.loc[temp_df.index[i],'sample',[key]]] += ':'+temp_df.loc[temp_df.index[i],key]
                    
                    for key in ['site','azimuth','dip','bed_dip_direction','bed_dip']: # data checks. no changes to file
                        for i in range(len(temp_df)):
                            if not temp_df.loc[temp_df.index[i],key] in samples_df.loc[samples_df['sample']==temp_df.loc[temp_df.index[i],'sample'],[key]]:
                                print('!WARNING! Sample '+temp_df.loc[temp_df.index[i],'sample']+' has multiple values for '+key+'!')
                                                    
            except:
                samples_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))
                
                
        elif magicfile == 'specimens.txt':
            #specimens_dict = {'specimen': [], 'sample': [],'method_codes': [],'experiments': [],'volume': [],
            #   'citations': [],'result_quality':[]}
            try:
                unique_specimens = list(set(specimens_df['specimen'])) # check if df exists
                temp_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))
                if len(temp_df[temp_df['specimen'].isin(unique_specimens)]) == 0:
                    specimens_df = pd.concat([specimens_df,temp_df],ignore_index=True)
                else: 
                    print('! check data for specimen '+ ','.join(list(temp_df['specimen']))+'! Did you do multiple experiments on it?')
                    specimens_df = pd.concat([specimens_df,temp_df[~temp_df['specimen'].isin(unique_specimens)]],ignore_index=True)
                    temp_df = temp_df[temp_df['specimen'].isin(unique_specimens)]                                                

                    for key in ['method_codes','citations','experiments']:
                        for i in range(len(temp_df)):
                            if not temp_df.loc[temp_df.index[i],key] in specimens_df.loc[specimens_df['specimen']==temp_df.loc[temp_df.index[i],'specimen'],[key]]:
                                specimens_df.loc[specimens_df['specimen']==temp_df.loc[temp_df.index[i],'specimen',[key]]] += ':'+temp_df.loc[temp_df.index[i],key]
                    
                    for key in ['sample','volume']: # data checks. no changes to file
                        for i in range(len(temp_df)):
                            if not temp_df.loc[temp_df.index[i],key] in specimens_df.loc[specimens_df['specimen']==temp_df.loc[temp_df.index[i],'specimen'],[key]]:
                                print('!WARNING! Specimen '+temp_df.loc[temp_df.index[i],'specimen']+' has multiple values for '+key+'!')
             
            except:
                specimens_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))
                
                                                 
        elif magicfile == 'measurements.txt':
            try: # all entries should be combined(?)
                len(measurements_df)
                temp_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))   
                if not len(temp_df[temp_df['specimen'].isin(list(set(measurements_df['specimen'])))]) == 0:
                     print('!WARNING! Specimen(s) '+''.join(temp_df[temp_df['specimen'].isin(list(set(measurements_df['specimen'])))])+' seem(s) to exist in multiple batches!')
                measurements_df = pd.concat([measurements_df,temp_df],ignore_index=True)                
            except:
                measurements_df = pd.read_csv(link+magicfile,skiprows=1,delimiter=('\t'))
                
                
        elif magicfile == 'locations.txt':
            print('! locations file found. Combine manually! I can\'t be arsed')
        else: print('! ignoring file '+ str(magicfile) + ' in ' + str(link))

print('! data combined. cleaning up files...\n')
os.chdir(save_dir)                                                 
with open('./sites.txt', 'w') as sites_file:
    sites_file.write('tab\tsites\n')
for i in range(len(sites_df)):
    for key in sites_df.keys():
        try:
            if ':' in sites_df.loc[sites_df.index[i],key]: 
                sites_df.loc[sites_df.index[i],key] = ':'.join(list(set(sites_df.loc[sites_df.index[i],key].split(':'))))
        except: continue
sites_df.to_csv('./sites.txt',mode='a',index=False,sep='\t')
    
with open('./samples.txt', 'w') as samples_file:
    samples_file.write('tab\tsamples\n')
for i in range(len(samples_df)):
    for key in samples_df.keys():
        try:
            if ':' in samples_df.loc[samples_df.index[i],key]: 
                samples_df.loc[samples_df.index[i],key] = ':'.join(list(set(samples_df.loc[samples_df.index[i],key].split(':'))))
        except: continue    
samples_df.to_csv('./samples.txt',mode='a',index=False,sep='\t')

with open('./specimens.txt', 'w') as specimens_file:
    specimens_file.write('tab\tspecimens\n')
for i in range(len(specimens_df)):
    for key in specimens_df.keys():
        try:
            if ':' in specimens_df.loc[specimens_df.index[i],key]:
                specimens_df.loc[specimens_df.index[i],key] = ':'.join(list(set(specimens_df.loc[specimens_df.index[i],key].split(':'))))
        except: continue
specimens_df.to_csv('./specimens.txt',mode='a',index=False,sep='\t')

with open('./measurements.txt', 'w') as measurements_file:
    measurements_file.write('tab\tmeasurements\n')
measurements_df = measurements_df[(~measurements_df['specimen'].isna())]
for i in range(len(measurements_df)):
    for key in measurements_df.keys():
        try:
            if ':' in measurements_df.loc[measurements_df.index[i],key]: 
                measurements_df.loc[measurements_df.index[i],key] = ':'.join(list(set(measurements_df.loc[measurements_df.index[i],key].split(':'))))
                
        except: continue
measurements_df.to_csv('./measurements.txt',mode='a',index=False,sep='\t')

print('Files for pmagpy GUI saved to', os.getcwd())

# Ignore everything below unless you know what you are doing!




JR6 -> Liverpool format for Shaw/pseudo-Thellier (PTSD-protocol)

In [ ]:
path = os.getcwd()
file = 'PI-PTSD-pottery-pilot-edit.jr6'
volume = 7 #cc

meta = {
    'section_code': 'TS',
    'measurer': 'Daniele Thallner',
    'density': 2.5,
    'experiment': 'AFTH',
    'bias': 60, #TRM bias field (uT)
    'temp': 620, #TRM max temp (C)
    'ARM_bias': 40, #ARM bias field (uT)
    'comment': '',
    'lat': 29.6, #GD: 45.9 , SC: 50, VT: 51, BB: 56, NW: 65.2
    'lon': 278, #GD: 280.0, SC: 302, VT: 24, BB: 356, NW: 234
    'ignore_chars': 0, #number of characters at the start of sample names that will be ignored
    'step': 'NRM0',  # type of the first treatment in file (has to be in steps_all)
    'steps_all': ['NRM0','ACQ0','ARM0','TRM1','ACQ1','ARM1','TRM2','ARM2'] # used protocol #,'ACQ0','ACQ1'
    }   

if not path.endswith('/'): path += '/'
f = open(path+file)
file = f.readlines()
f.close()
for line in file:
    line = line[:9].strip(' ') + line[9:].replace('-',',-').replace(' ',',').replace('\t',',')
    while ',,' in line:
        line=line.replace(',,',',')
    with open('temp.txt','a') as tempfile:
        tempfile.write(line)
f = open('temp.txt')
file = f.readlines()
f.close()
os.remove('temp.txt')
samplelist = []
for i in range(len(file)):
    file[i] = file[i].split(',')
    if not file[i][0] in samplelist: 
        samplelist.append(file[i][0])
        with open('temp.txt','a') as tempfile:
            tempfile.write(','.join(file[i]))
        for j in range(i+1,len(file)):
            temp = file[j].split(',')
            if temp[0] == samplelist[-1]:
                with open('temp.txt','a') as tempfile:
                    tempfile.write(','.join(temp))
f = open('temp.txt')
file = f.readlines()
f.close()
os.remove('temp.txt')

pthells = []
for i in range(len(file)):
    file[i] = file[i].strip('\n').split(',')
    if i == 0 or not file[i][0] == file[i-1][0]:
        header, body, end = {},{'a':[]},{'a':'END'}
        step, stepnr = 0, 0
        if len(file[i][0]) > 9:
                file[i][0] = file[i][0][-10:]
        header['a'] = file[i][0][meta['ignore_chars']:]#+'.'+meta['steps_all'][step] #'sample_code'
        header['b'] = 0 #'dip'
        header['c'] = 90 #'yetn'
        header['d'] = 0 #'height'
        header['e'] = 0 #'position'
        if np.isnan(header['e']) :
            header['e'] = 0
        header['f'] = 0 #thickness
        header['g'] = 0 #unit dip
        if np.isnan(header['g']) :
            header['g'] = 0
        header['h'] = 0 # unit dir
        if np.isnan(header['h']) :
            header['h'] = 360
        header['i'] = meta['lat']#site lat
        header['j'] = meta['lon']#site lon
        if meta['step'][0] == 'T':
            header['k'] = 'THAF-'+meta['experiment']
        else: header['k'] = 'AF-D'   
        header['l'] = ''+meta['measurer'] #analyst
        header['m'] = 'JR6' #magnetometer
        header['n'] = 'AF' #demagnetiser
        header['o'] = '' #comment
        if not type(header['o']) == str:
            if np.isnan(header['o']):
                header['o'] = 0 
        header['p'] = '!!!NOT UPLOADED!!!'
        header['q'] = '!!!NOT CONVERTED!!!'
        header['r'] = volume #volume
        header['s'] = meta['density']*1000 #density
    if i >= 0:
        if (len(body['a']) > 1 and not file[i][1][:2] == file[i-1][1][:2]) and not stepnr == 1:
            step = step+1
            stepnr = 0
        #elif 'AD' in file[i][1]and 'AD' in file[i-1] and float(file[i][1].strip('AD')) < float(file[i-1][1].strip('AD')):
        #    step = step+1 # if the file consists of more sets of AF measurements
        #    stepnr = 0
        #if len(file[i][-1]) < 10: print('ERROR! Check the jr6 file of', header['a'],'(probably fucked up import)')
        
        if 'NRM' in file[i][1]: tempfield = 0
        elif 'AD' in file[i][1]: tempfield = int(float(file[i][1].strip('AD')))
        elif 'ARM' in file[i][1]: tempfield = int(float(file[i][1].strip('ARM')))
        elif 'TRM' in file[i][1]: tempfield = int(float(file[i][1].strip('TRM')))
        else: tempfield = 0
        try: body['a'].append(tempfield) # temperature/field
        except: body['a'] = [tempfield]
        
        try: body['b'].append(0) # MW power
        except: body['b'] = [0]
        try: body['c'].append(0) # MW time
        except: body['c'] = [0]
        try: body['d'].append(float(file[i][2])*10**int(file[i][5])) #x (nAm2) A/m!
        except: body['d'] = [float(file[i][2])*10**int(file[i][5])]
        try: body['e'].append(float(file[i][3])*10**int(file[i][5])) #y (nAm2) A/m!
        except: body['e'] = [float(file[i][3])*10**int(file[i][5])]
        try: body['f'].append(float(file[i][4])*10**int(file[i][5])) #z (nAm2) A/m!
        except: body['f'] = [float(file[i][4])*10**int(file[i][5])]
        try: body['g'].append(float(header['r'])*float(header['s'])/1000) #mass
        except: body['g'] = [float(header['r'])*float(header['s'])/1000]
        try: body['h'].append(meta['ARM_bias']) #applied field intensity uT
        except: body['h'] = [meta['ARM_bias']]
        try: body['i'].append(0) #applied field dec
        except: body['i'] = [0]
        if body['h'][-1] > 0:
            try: body['j'].append(90) #applied field inc
            except: body['j'] = [90]
        else:
            try: body['j'].append(0) #applied field inc
            except: body['j'] = [0]
        datetime = ''#file[i][-1].split(' ')
        try: body['k'].append(datetime) #measurement date
        except: body['k'] = [datetime]
        try: body['l'].append(datetime) #measurement time
        except: body['l'] = [datetime] # if len(datetime[1]) < 0 ?
        try: body['m'].append(meta['steps_all'][step]) #comment
        except: 
            body['m'] = [meta['steps_all'][step]]
        if 'TRM1' in meta['steps_all'][step] or 'TRM2' in meta['steps_all'][step]:
            header['o'] = 'Shaw-TRM with '+ str(meta['bias']) +'uT at '+ str(meta['temp']) +'C'
        elif 'ACQ0' in meta['steps_all'][step]:
            header['o'] = 'pseudo-Thellier experiment'
        if ('ACQ0' in meta['steps_all'][step] or 'ACQ1' in meta['steps_all'][step]) and not file[i][0] in pthells: pthells.append(file[i][0])
        try: body['n'].append(stepnr) #step number
        except: body['n'] = [stepnr]
        stepnr+=1
        if 'NRM' in file[i][1]: 
            try: body['o'].append('NRM') #steptype
            except: body['o'] = ['NRM']
        else:
            if 'ARM' in file[i][1]:
                try: body['o'].append('I') #steptype
                except: body['o'] = ['I']
            else:
                try: body['o'].append('Z') #steptype
                except: body['o'] = ['Z']
                body['h'][-1] = 0
        try: body['p'].append(0) #tristan gain
        except: body['p'] = [0]
        try: body['q'].append(0) #MW integral
        except: body['q'] = [0]
        try: body['r'].append(file[i][-1]) #jr6 error
        except: body['r'] = [file[i][-1]]
        try: body['s'].append(file[i][-1]) #Fit
        except: body['s'] = [file[i][-1]]
        try: body['t'].append(file[i][-1]) #Utrecht error
        except: body['t'] = [file[i][-1]]
        if 'TRM' in file[i][1]:
            try: body['u'].append(0) #af peak field
            except: body['u'] = [0]
        else:
            try: body['u'].append(tempfield) #af peak field
            except: body['u'] = [tempfield]
        if 'TRM' in file[i][1]:
            try: body['v'].append(tempfield) #th peak field
            except: body['v'] = [tempfield]
        else:
            try: body['v'].append(0) #th peak field
            except: body['v'] = [0]
                        #print(header['a'],file[i][0],body['m'][-1])
        if len(body['a'])>1:
            if body['a'][-1] == body['a'][-2] and body['m'][-1] == body['m'][-2]:
                print('! double step found for sample',header['1'])
                print(file[i])
    if i < len(file)-1 or i == len(file)-1:
        if i == len(file)-1 or not file[i][0] == file[i+1].split(',')[0]:
            head = pd.DataFrame(header, index = [0])
            head.loc[head.index[0], 'a'] = head.loc[head.index[0], 'a']
            head.to_csv('./'+meta['section_code']+'_PTSD.csv', index = False, header=False, mode = 'a')
            body_df = pd.DataFrame(body)
            body_df.to_csv('./'+meta['section_code']+'_PTSD.csv', index = False, header=False, mode = 'a')
            end_df = pd.DataFrame(end, index = [0])
            end_df.to_csv('./'+meta['section_code']+'_PTSD.csv', index = False, header=False, mode = 'a')
            print(header['a'] + ' converted! \(✿◠‿◠)/')

In [ ]:
def remove_residual(file,mode):
    f = open(file)
    lines = f.readlines()
    f.close()
    h = open('./temp.csv',mode='a')
    datalines = []
    for line in lines:
        line = line.strip(' ').split(',')
        try: 
            float(line[20].strip(' '))
            datalines.append(line)
            if line[12] == mode:
                nx = float(datalines[-1][3])
                ny = float(datalines[-1][4])
                nz = float(datalines[-1][5])
                px, py, pz = 0,0,0
            elif line[12] == 'ACQ'+str(mode[-1]) and px == 0 and py == 0 and pz == 0:
                px = float(datalines[-1][3])
                py = float(datalines[-1][4])
                pz = float(datalines[-1][5])
            elif line[12] == 'ARM' + str(mode[-1]):
                ax = float(datalines[-1][3])
                ay = float(datalines[-1][4])
                az = float(datalines[-1][5])
        except:
            if not 'END' in line[0]:
                templine = ''
                for i in range(len(line)):
                    templine += line[i].strip(' ') + ','
                h.write(templine[:-1])
                datalines = []
            else:
                for entry in datalines:
                    if entry[12] == mode:
                        entry[3] = float(entry[3]) - nx
                        entry[4] = float(entry[4]) - ny
                        entry[5] = float(entry[5]) - nz
                    elif entry[12] == 'ACQ'+str(mode[-1]):
                        entry[3] = float(entry[3]) - px
                        entry[4] = float(entry[4]) - py
                        entry[5] = float(entry[5]) - pz
                    elif entry[12] == 'ARM' + str(mode[-1]):
                        entry[3] = float(entry[3]) - ax 
                        entry[4] = float(entry[4]) - ay
                        entry[5] = float(entry[5]) - az
                    templine = ''
                    for i in range(len(entry)):
                        templine += str(entry[i]).strip(' ') + ','
                    h.write(templine[:-1])
    print('Residuals have been removed! (づ￣ ³￣)づ')
    h.close()
    
def pTH_online(file, noresid = True,mode='NRM0'):
    if noresid:
        remove_residual(file,mode)
        f = open('./temp.csv')
        lines = f.readlines()
        f.close()
        os.remove('./temp.csv')
        h = open('./'+file[:6]+'upload_noResidual.txt',mode='a')
    else:
        f = open(file)
        lines = f.readlines()
        f.close()
        h = open('./'+file[:6]+'upload.txt',mode='a')
    text = 'specimen\tstep\tdec_s\tinc_s\tmoment\ttype\n'
    h.write(text)
    #name,vol = '',0
    name = ''
    for line in lines:
        text = ''
        line = line.split(',')
        try:
            step = float(line[20])
            if line[12] == mode or line[12] == 'ACQ'+str(mode[-1]) or line[12] == 'ARM' + str(mode[-1]):
                #step = tmp/10
                #if not float(line[3]) == 0 and not float(line[4]) == 0 and not float(line[5]) == 0:
                try:
                    if not float(line[3]) == 0 and not float(line[4]) == 0:
                        dec = (np.arctan2(float(line[4]),float(line[3]))*180/np.pi)%360
                    else:
                        dec = tempdec
                    #if((float(line[3]) < 0 and float(line[4]) > 0) or (float(line[3]) < 0 and float(line[4]) < 0)):
                    #    dec = dec + 180
                    #elif(float(line[3]) > 0 and float(line[4]) < 0):
                    #    dec = dec + 360
                    if not float(line[5]) == 0:
                        inc = np.arctan2(float(line[5]),np.sqrt(float(line[3])**2+float(line[4])**2))*180/np.pi
                    else:
                        inc = tempinc
                    moment = np.sqrt(float(line[3])**2+float(line[4])**2+float(line[5])**2)# * vol * 10**6 # emu/cc to nAm2 (convert PTSD already converts emu to nAm2)
                    tempdec, tempinc = dec, inc
                #else:
                except:
                    dec, inc, moment = tempdec,tempinc,0
                    print(line)
                if line[12] == mode:
                    steptype = 0
                elif line[12] == 'ACQ'+str(mode[-1]):
                    steptype = 6
                elif line[12] == 'ARM'+str(mode[-1]):
                    steptype = 7
                text += name + '\t' + str(step) + '\t' + str(dec) +'\t' + str(inc) + '\t' + str(moment) +'\t' + str(steptype)+'\n'
        except:
            if not 'END' in line[0]:
                name = line[0].strip(' ')
                #vol = float(line[17])
        h.write(text)
        text = ''
    h.close()
    print('╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ paleointensity.org - pseudoThellier-uploadfile (pth format) created!')


In [ ]:
start = 'NRM0'

if len(pthells) == 0: print('No specimens to convert')
f = open('./'+meta['section_code']+'_PTSD.csv')
ptsdfile = f.readlines()
f.close()
header,body, = {},{}
    
pth = False
for line in ptsdfile:
    line = line.split(',')
    if line[0] in pthells:
        pth = True
        for i in range(len(line)):
            header[str(i)] = line[i].strip('\n')
        head = pd.DataFrame(header, index = [0])
        head.to_csv('./pth_temp.csv', index = False, header=False, mode = 'a')
    elif 'END' in line[0]:
        end_df.to_csv('./pth_temp.csv', index = False, header=False, mode = 'a')
        pth = False
    else:
        if pth == True:
            for i in range(len(line)):
                body[str(i)] = line[i].strip('\n')
                if 'TRM' in line[12]: body['0'] = 0
            body_df = pd.DataFrame(body, index = [0])
            body_df.to_csv('./pth_temp.csv', index = False, header=False, mode = 'a')
            
pTH_online('./pth_temp.csv', noresid = True, mode=start)
os.remove('./pth_temp.csv')